<a href="https://colab.research.google.com/github/bebyakinb/test_assignments_Analyst/blob/master/habr/SQL_Hardest_Interview_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Подготовка среды

In [3]:
!wget https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release -O sqlite.tar.gz
!tar xzf sqlite.tar.gz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.8/pysqlite3/_sqlite3.cpython-38-x86_64-linux-gnu.so \
    /usr/lib/python3.8/lib-dynload/_sqlite3.cpython-38-x86_64-linux-gnu.so

--2022-12-08 12:10:06--  https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release
Resolving www.sqlite.org (www.sqlite.org)... 45.33.6.223, 2600:3c00::f03c:91ff:fe96:b959
Connecting to www.sqlite.org (www.sqlite.org)|45.33.6.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12076208 (12M) [application/x-compressed]
Saving to: ‘sqlite.tar.gz’

sqlite.tar.gz       100%[===================>]  11.52M  26.9MB/s    in 0.4s    

2022-12-08 12:10:07 (26.9 MB/s) - ‘sqlite.tar.gz’ saved [12076208/12076208]

/content/sqlite
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
chec

In [1]:
import sqlite3
sqlite3.sqlite_version

'3.40.0'

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Задача №1. Процентное изменение месяц к месяцу


## 1.0. Постановка задачи
**Контекст:** часто полезно знать, как изменяется ключевая метрика, например, месячная аудитория активных пользователей, от месяца к месяцу. Допустим у нас есть таблица logins в таком виде:
```
| user_id | date       |
|---------|------------|
| 1       | 2018-07-01 |
| 234     | 2018-07-02 |
| 3       | 2018-07-02 |
| 1       | 2018-07-02 |
| ...     | ...        |
| 234     | 2018-10-04 |
```
**Задача:** Найти ежемесячное процентное изменение месячной аудитории активных пользователей (MAU).

## 1.1. Подготовка данных

In [61]:
def random_dates(start, end, n=10):
    '''
    Dividing the unix time values(start and end) by 24*60*60*10**9
    to make it days and return n randomly chosen days back.
    '''
    start_u = start.value//(24*60*60*10**9)
    end_u = end.value//(24*60*60*10**9)

    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='D')


In [174]:
start = pd.to_datetime('2021-01-01')
end = pd.to_datetime('2022-12-01')
login_lines = 100000
user_ids = 3000

logins = pd.DataFrame()
logins['user_id'] = np.random.randint(1, user_ids, login_lines)
logins['date'] = pd.Series(random_dates(start, end, login_lines))
logins

,user_id,date
0,1929,2022-08-19
1,1859,2022-03-22
2,89,2022-10-23
3,1500,2021-08-15
4,2120,2021-09-29
...,...,...
99995,2429,2022-04-21
99996,1164,2021-09-18
99997,756,2021-02-02
99998,1449,2021-04-25


In [5]:
con = sqlite3.connect('db')
cur = con.cursor()

In [6]:
logins.to_sql('logins', con, if_exists='replace',index=False)

In [7]:
def select(sql):
  return pd.read_sql(sql,con)

In [8]:
sql = '''select * from logins'''
select(sql)

,user_id,date
0,234,2021-01-22 00:00:00
1,5,2022-07-23 00:00:00
2,154,2022-01-20 00:00:00
3,28,2022-03-31 00:00:00
4,184,2021-02-23 00:00:00
...,...,...
99995,37,2021-11-15 00:00:00
99996,57,2022-07-23 00:00:00
99997,109,2021-04-27 00:00:00
99998,46,2022-04-13 00:00:00


## 1.2. Решение SQL

In [58]:
sql ='''
with months_active as (
  select strftime('%Y %m', date) as year_month,
         cast(count(user_id) as real) as users from logins
  group by year_month)

select 
year_month, users, (users - LAG(users) over(order by year_month))*100/LAG(users) over(order by year_month) MAU_percent
from months_active
'''
select(sql)

,year_month,users,MAU_percent
0,2021 01,4462.0,NaN
1,2021 02,4004.0,-10.264455
2,2021 03,4461.0,11.413586
3,2021 04,4266.0,-4.371217
4,2021 05,4335.0,1.617440
5,2021 06,4235.0,-2.306805
6,2021 07,4468.0,5.501771
7,2021 08,4448.0,-0.447628
8,2021 09,4217.0,-5.193345
9,2021 10,4429.0,5.027271


## 1.3 Решение Pandas

In [183]:
pt = logins.pivot_table(index=[(logins['date'].dt.to_period('M'))], values=['user_id'], aggfunc="count")
pt['MAU,%mm'] = (pt['user_id'] - pt['user_id'].shift(1))/pt['user_id'].shift(1)*100
pt

,user_id,"MAU,%mm"
date,,
2021-01,4509,NaN
2021-02,3976,-11.820803
2021-03,4689,17.932596
2021-04,4216,-10.087439
2021-05,4454,5.645161
2021-06,4281,-3.884149
2021-07,4408,2.966597
2021-08,4503,2.155172
2021-09,4353,-3.331113


# 2.Задача №2. Маркировка древовидной структуры

## 2.0 Постановка задачи
Контекст: предположим, у вас есть таблица tree с двумя столбцами: в первом указаны узлы, а во втором — родительские узлы.
```
node   parent
1       2
2       5
3       5
4       3
5       NULL 
```
Задача: написать SQL таким образом, чтобы мы обозначили каждый узел как внутренний (inner), корневой (root) или конечный узел/лист (leaf), так что для вышеперечисленных значений получится следующее:
```
node    label  
1       Leaf
2       Inner
3       Inner
4       Leaf
5       Root
```

## 2.1 Подготовка данных

In [197]:
nodes = pd.DataFrame({'node': [1,2,3,4,5],
                      'parent':[2,5,5,3,np.nan]})
nodes

,node,parent
0,1,2.0
1,2,5.0
2,3,5.0
3,4,3.0
4,5,NaN


In [198]:
nodes.to_sql('nodes', con, if_exists='replace',index=False)

### 2.2 Решение SQL

In [199]:
sql = '''
select node,
       case
            when parent is null then 'Root'
            when node in (select parent from nodes) then 'Inner'
            else 'Leaf'
       end as label
from nodes
'''
select(sql)

,node,label
0,1,Leaf
1,2,Inner
2,3,Inner
3,4,Leaf
4,5,Root


## 2.3 Решение Pandas

In [120]:
nodes['label'] =  np.where(nodes['parent'].isna(), 'Root',
                  np.where(nodes['node'].isin(nodes['parent']), 'Inner', 'Leaf'))
nodes[['node','label']]

,node,label
0,1,Leaf
1,2,Inner
2,3,Inner
3,4,Leaf
4,5,Root


# 3. Задача № 3. Удержание пользователей в месяц.

## 3.0. Постановка задачи

**Контекст:** допустим, у нас есть статистика по авторизации пользователей на сайте в таблице logins:
```
| user_id | date       |
|---------|------------|
| 1       | 2018-07-01 |
| 234     | 2018-07-02 |
| 3       | 2018-07-02 |
| 1       | 2018-07-02 |
| ...     | ...        |
| 234     | 2018-10-04 |
```

**Задача:** написать запрос, который получает количество удержанных пользователей в месяц. В нашем случае данный параметр определяется как количество пользователей, которые авторизовались в системе и в этом, и в предыдущем месяце.


## 3.1. Подготовка данных

In [222]:
logins.to_sql('logins', con, if_exists='replace',index=False)

## 3.2. Решение SQL

In [221]:
sql = '''
with months_logins as (
  select distinct user_id, 
         strftime('%Y %m', date) as year_month,
         strftime('%Y %m', date(date, 'start of month','-1 month')) as prev_month
  from logins
  )

select m1.year_month, count(m2.user_id)
from months_logins as m1
left join months_logins as m2 ON m1.user_id = m2.user_id AND m1.prev_month = m2.year_month
group by m1.year_month
'''
select(sql)

,year_month,count(m2.user_id)
0,2021 01,0
1,2021 02,1688
2,2021 03,1718
3,2021 04,1756
4,2021 05,1761
5,2021 06,1765
6,2021 07,1743
7,2021 08,1779
8,2021 09,1760
9,2021 10,1767


## 3.3. Решение Pandas

In [230]:
logins_extended = logins.copy()
logins_extended['id_and_month'] = (
    logins['user_id'].astype(str) 
    + '_' 
    + logins['date'].dt.to_period('M').dt.to_timestamp().astype(str))
logins_extended['id_and_prev_month'] = (
    logins['user_id'].astype(str) 
    + '_' 
    + (logins['date'].dt.to_period('M').dt.to_timestamp() 
       - pd.DateOffset(months=1)).astype(str))
logins_extended = logins_extended.drop_duplicates(['user_id','id_and_month'])
logins_extended

,user_id,date,id_and_month,id_and_prev_month
0,1929,2022-08-19,1929_2022-08-01,1929_2022-07-01
1,1859,2022-03-22,1859_2022-03-01,1859_2022-02-01
2,89,2022-10-23,89_2022-10-01,89_2022-09-01
3,1500,2021-08-15,1500_2021-08-01,1500_2021-07-01
4,2120,2021-09-29,2120_2021-09-01,2120_2021-08-01
...,...,...,...,...
99991,2186,2021-05-12,2186_2021-05-01,2186_2021-04-01
99995,2429,2022-04-21,2429_2022-04-01,2429_2022-03-01
99996,1164,2021-09-18,1164_2021-09-01,1164_2021-08-01
99997,756,2021-02-02,756_2021-02-01,756_2021-01-01


In [234]:
logins_extended['prev_month_login'] = (
    np.where(logins_extended['id_and_prev_month'].isin(logins_extended['id_and_month'])
             , 1 , 0))
logins_extended.groupby([logins_extended['date'].dt.to_period('M')])['prev_month_login'].sum()

date
2021-01       0
2021-02    1688
2021-03    1718
2021-04    1756
2021-05    1761
2021-06    1765
2021-07    1743
2021-08    1779
2021-09    1760
2021-10    1767
2021-11    1754
2021-12    1756
2022-01    1814
2022-02    1748
2022-03    1713
2022-04    1743
2022-05    1731
2022-06    1760
2022-07    1794
2022-08    1818
2022-09    1773
2022-10    1747
2022-11    1760
Freq: M, Name: prev_month_login, dtype: int64